### Project 4: Continued 
In the previous notebook, I did all the processing I could within the memory limitations of my laptop. I exported the cleaned CSV and starting with a clean kernel to One Hot encode the data so that the customers are the index and each column is a movie. I will also explore PCA. 

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
#read the previously cleaned csv
df = pd.read_csv('combined_data_cleaned.csv')
df = df.astype('uint32')

### Continuing to Reduce Data Size 
Need to reduce size of data still. In the previous notebook, somewhat methodical methods of compression were implemented. The data was reduced a considerable amount. But it is still too large for some of the operations I want to complete. At this point we'll randomly drop customers and movies using the function below.

In [3]:
import typing as T
import random as rand

#Drop random indices from given list
def drop_random(values: list, length: int, proportion: float):
    '''
    Parameters
        values: "list like" - Array of original values before randomly dropped
        length: int - length of the values array
        proportion: float - proportion of values to KEEP ex. .8 keep 80% of the data
    '''
    num_to_drop = length -  int(length*proportion)
    to_drop = np.empty(num_to_drop, dtype=int)
    
    for i in range(0,num_to_drop):
        while True:
            n = rand.randrange(length)
            if n not in to_drop[:]:
                to_drop[i] = n
                break

    return np.delete(values, to_drop, axis=None)

In [4]:
#get customer and movie unqique values
customers = df.Cust_Id.value_counts().index
movies = df.Movie_Id.value_counts().index

# get random customers and movies to drop
customers_to_drop = drop_random(customers, len(customers), .1)
movies_to_drop = drop_random(movies, len(movies), .3)

#drop the random customers and movies
df_trimmed = df[df['Cust_Id'].isin(customers_to_drop)]
df_trimmed = df_trimmed[df_trimmed['Movie_Id'].isin(movies_to_drop)]
del df

In [5]:
print("Customer count before drop:\t", len(customers))
print("Movie count before drop:\t", len(movies))
print("Customer count after drop:\t", len(df_trimmed.Cust_Id.value_counts().index))
print("Movie count after drop:\t\t", len(df_trimmed.Movie_Id.value_counts().index))

Customer count before drop:	 291590
Movie count before drop:	 16031
Customer count after drop:	 29159
Movie count after drop:		 4789


In [6]:
df_trimmed.info(verbose=False, memory_usage="deep")

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1065779 entries, 1575 to 35021621
Columns: 3 entries, Cust_Id to Movie_Id
dtypes: uint32(3)
memory usage: 20.3 MB


In [7]:
#rearrange data frame so that customers and rows and movies are columns
df_pivot = df_trimmed.pivot_table(index = 'Cust_Id', columns = 'Movie_Id', values = 'Rating')
del df_trimmed

In [8]:
df_pivot.shape

(29159, 4789)

In [9]:
#export reformatted data 
df_pivot.to_csv('data_pivot.csv')